In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
spark_context = SparkContext("local", "first_app")
spark = SparkSession(spark_context)
import plotly.graph_objects as go
from pyspark.sql import SQLContext
sql_context = SQLContext(spark_context)

In [2]:
text_file = sql_context.read.format("com.databricks.spark.csv").options(header='true', inferschema='true', quote='"', delimiter=',').load("/home/jovyan/work/Covid19.csv")
rdd_filter = text_file.rdd.map(tuple)
rdd_covid = rdd_filter.map(lambda word: (word[2], word[5]))
rdd_covid.take(5)

[(9, 0), (9, 125), (9, 47), (9, 0), (9, 17)]

In [3]:
rdd_covid = rdd_covid.reduceByKey(lambda a,b: a+b)
print(rdd_covid.collect())

[(9, 2374517), (8, 3584040), (7, 3288723), (6, 2527885), (5, 1555025), (3, 324799), (4, 868927), (2, 73635), (1, 9758)]


In [4]:
rdd_covid = spark_context.parallelize(rdd_covid.sortBy(lambda a: a[1], True).take(5))
print(rdd_covid.collect())

[(1, 9758), (2, 73635), (3, 324799), (4, 868927), (5, 1555025)]


In [5]:
covid_month = rdd_covid.map(lambda x: x[0])
covid_cases = rdd_covid.map(lambda x: x[1])
print(covid_month.collect())
print(covid_cases.collect())

[1, 2, 3, 4, 5]
[9758, 73635, 324799, 868927, 1555025]


In [6]:
fig = go.Figure(data=go.Bar(x=covid_month.collect(),y=covid_cases.collect()))
fig.update_layout(title_text='Total cases per month',title_font_size=30,
                  yaxis=dict(title='Cases',title_font_size=25),
                  xaxis=dict(title='Month',title_font_size=25))
fig.update_traces(overwrite=True, marker={"opacity": 0.5})
fig.write_html('report6.html', auto_open=True)